In [12]:
using BenchmarkTools

In [13]:
findAllFactors(n::Integer) = filter(k -> n % k == 0, 1:n)

findAllFactors (generic function with 1 method)

In [14]:
@btime findAllFactors(2^4*5^3*3^7)

  7.979 ms (5 allocations: 33.37 MiB)


160-element Vector{Int64}:
       1
       2
       3
       4
       5
       6
       8
       9
      10
      12
       ⋮
  437400
  486000
  546750
  729000
  874800
 1093500
 1458000
 2187000
 4374000

In [15]:
isPrime(n::Integer) = length(findAllFactors(n)) == 2

isPrime (generic function with 1 method)

In [16]:
nextPrime(n::Integer) = isPrime(n+1) ? n+1 : nextPrime(n+1)

nextPrime (generic function with 1 method)

In [17]:
nextPrime(16), nextPrime(26), nextPrime(1_000_000)

(17, 29, 1000003)

In [4]:
isPrime(11)

true

In [56]:
getPrimes(n) = filter(isPrime,2:n)

getPrimes (generic function with 2 methods)

In [19]:
getPrimes(100)

25-element Vector{Int64}:
  2
  3
  5
  7
 11
 13
 17
 19
 23
 29
  ⋮
 59
 61
 67
 71
 73
 79
 83
 89
 97

In [40]:
function getPrimes2(n::Integer)
  local primes = Int[]
  local k = 2
  while k < n
    push!(primes, k)
    k = nextPrime(k)
  end
  primes
end

getPrimes2 (generic function with 1 method)

In [25]:
getPrimes2(100)

25-element Vector{Int64}:
  2
  3
  5
  7
 11
 13
 17
 19
 23
 29
  ⋮
 59
 61
 67
 71
 73
 79
 83
 89
 97

In [27]:
function isPerfect(n::Integer)
  A=findAllFactors(n)
  pop!(A)
  sum(A)==n
end


isPerfect (generic function with 1 method)

In [28]:
isPerfect2(n::Integer) = sum(findAllFactors(n)) == 2n

isPerfect2 (generic function with 1 method)

In [29]:
@time isPerfect(100_000)

  0.000465 seconds (5 allocations: 781.672 KiB)


false

In [30]:
@time isPerfect2(100_000)

  0.000187 seconds (5 allocations: 781.672 KiB)


false

In [31]:
@btime isPerfect(100_000)

  180.292 μs (5 allocations: 781.67 KiB)


false

In [32]:
@btime isPerfect2(100_000)

  181.000 μs (5 allocations: 781.67 KiB)


false

In [2]:
digits(1234)

4-element Vector{Int64}:
 4
 3
 2
 1

In [36]:
function isHappy(n::Integer)
  if n==1
    return true
  elseif n==4
    return false
  else
    local d = digits(n)
    local sum = 0
    for i=1:length(d)
      sum += d[i]^2
    end
    return isHappy(sum)
  end
end

isHappy (generic function with 1 method)

In [35]:
function isHappy2(n::Integer)
  if n==1
    return true
  elseif n==4
    return false
  else
    return isHappy2(sum(x->x^2,digits(n)))
  end
end


isHappy2 (generic function with 1 method)

In [33]:
isHappy3(n::Integer) = n == 1 ? true : n == 4 ? false : isHappy3(sum(x->x^2,digits(n)))

isHappy3 (generic function with 1 method)

In [4]:
filter(isHappy3, 1:100)

20-element Vector{Int64}:
   1
   7
  10
  13
  19
  23
  28
  31
  32
  44
  49
  68
  70
  79
  82
  86
  91
  94
  97
 100

In [5]:
join(filter(isHappy3, 1:100)," ")

"1 7 10 13 19 23 28 31 32 44 49 68 70 79 82 86 91 94 97 100"

In [37]:
@btime isHappy(1_234)
@btime isHappy2(1_234)
@btime isHappy3(1_234)

  901.786 ns (24 allocations: 960 bytes)
  890.957 ns (24 allocations: 960 bytes)
  892.196 ns (24 allocations: 960 bytes)


false

In [6]:
n = big(2)^89-1

618970019642690137449562111

In [10]:
isPrime(n::Integer)= length(findAllFactors(n))==2

isPrime (generic function with 1 method)

In [11]:
@btime isPrime(1_000_003)

  1.098 ms (5 allocations: 7.63 MiB)


true

### Speedup #1

In [65]:
function findAllFactors2(n::Integer)
  factors = [1]
  for i=2:n-1
    if n % i ==0
      push!(factors,i)
    end
  end
  push!(factors,n) # n is always a factor of itself
end

findAllFactors2 (generic function with 1 method)

In [66]:
@btime findAllFactors(49_000_000)

  90.179 ms (5 allocations: 373.84 MiB)


147-element Vector{Int64}:
        1
        2
        4
        5
        7
        8
       10
       14
       16
       20
        ⋮
  3062500
  3500000
  4900000
  6125000
  7000000
  9800000
 12250000
 24500000
 49000000

In [67]:
@btime findAllFactors2(49_000_000)

  50.245 ms (5 allocations: 1.97 KiB)


147-element Vector{Int64}:
        1
        2
        4
        5
        7
        8
       10
       14
       16
       20
        ⋮
  3062500
  3500000
  4900000
  6125000
  7000000
  9800000
 12250000
 24500000
 49000000

In [68]:
isPrime2(n::Integer) = length(findAllFactors2(n))==2

isPrime2 (generic function with 1 method)

In [69]:
@btime isPrime2(1_000_003)

  1.023 ms (3 allocations: 160 bytes)


true

### Speedup #2: reducing the checked factors

In [70]:
function findAllFactors3(n::Integer)
  local factors = [1]
  for i=2:n÷2
    if n % i ==0
      push!(factors,i)
    end
  end
  push!(factors,n)
end

findAllFactors3 (generic function with 1 method)

In [71]:
@btime findAllFactors3(49_000_000)

  25.071 ms (5 allocations: 1.97 KiB)


147-element Vector{Int64}:
        1
        2
        4
        5
        7
        8
       10
       14
       16
       20
        ⋮
  3062500
  3500000
  4900000
  6125000
  7000000
  9800000
 12250000
 24500000
 49000000

In [72]:
isPrime3(n::Integer) = length(findAllFactors3(n)) == 2

isPrime3 (generic function with 1 method)

In [73]:
@btime isPrime3(1_000_003)

  511.250 μs (3 allocations: 160 bytes)


true

In [74]:
isPrime3(n::Integer) = length(findAllFactors3(n))==2

isPrime3 (generic function with 1 method)

In [75]:
@btime isPrime3(1_000_003)

  511.250 μs (3 allocations: 160 bytes)


true

#### Speedup #3: Notice that factors come in pairs 

In [76]:
function findAllFactors4(n::Integer)
  local x = round(Int,sqrt(n)) # closest integer to sqrt(n)
  local factors = [1,n]
  local j=2 # keep track where to insert elements
  for k=2:x
    if n%k==0
      # Insert the new factors in the middle of the factors array.
      # If k^2 is n, just add k, otherwise add the pairs.
      splice!(factors,j:(j-1),k^2 == n ? [k] : [k,div(n,k)])
      j+=1
    end
  end
  factors
end

findAllFactors4 (generic function with 1 method)

In [77]:
@btime findAllFactors4(49_000_000)

  14.541 μs (230 allocations: 15.73 KiB)


147-element Vector{Int64}:
        1
        2
        4
        5
        7
        8
       10
       14
       16
       20
        ⋮
  3062500
  3500000
  4900000
  6125000
  7000000
  9800000
 12250000
 24500000
 49000000

In [78]:
isPrime4(n::Integer) = length(findAllFactors4(n))==2

isPrime4 (generic function with 1 method)

In [79]:
@btime isPrime4(1_000_003)

  1.054 μs (2 allocations: 80 bytes)


true

#### Speedup #5: don't use factors at all

In [80]:
function isPrime5(n::Integer)
  for k=2:floor(Int,sqrt(n)) # integer nearest sqrt(n)
    if n%k==0
      return false
    end
  end
  true
end

isPrime5 (generic function with 1 method)

In [81]:
@btime isPrime5(1_000_003)

  1.012 μs (0 allocations: 0 bytes)


true

#### Speedup #6: skip all even numbers

In [82]:
function isPrime6(n::Integer)
  if n == 1
  	return false
  elseif n == 2
    return true
  elseif n%2==0
    return false
  end
  for k=3:2:floor(Int,sqrt(n)) # odd integers to sqrt(n)
    if n%k==0
      return false
    end
  end
  true
end

isPrime6 (generic function with 1 method)

In [83]:
@btime isPrime6(1_000_003)

  509.715 ns (0 allocations: 0 bytes)


true

In [84]:
nextPrime3(n::Integer) = isPrime6(n+1) ? n+1 : nextPrime3(n+1)

nextPrime3 (generic function with 1 method)

In [85]:
function getPrimes3(m::Integer) ## return all primes up to n using
  # the sieve of Eratosthenes
  local is_prime=trues(m) ## assume all are prime
  local k=2
  while k < sqrt(m)
    is_prime[2*k:k:m] .= false # all multiples of k are not prime
    k = nextPrime(k+1) # find the next prime after k
  end
  findall(is_prime)[2:end]
end

getPrimes3 (generic function with 1 method)

In [86]:
@btime getPrimes2(10_000)

  134.839 ms (49784 allocations: 384.57 MiB)


1229-element Vector{Int64}:
    2
    3
    5
    7
   11
   13
   17
   19
   23
   29
    ⋮
 9901
 9907
 9923
 9929
 9931
 9941
 9949
 9967
 9973

In [87]:
join(getPrimes(100), " ")

"2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97"

In [88]:
@btime getPrimes(1_000_000)
# @btime getPrimes2(10_000)
@btime getPrimes3(1_000_000)

  6.143 ms (10 allocations: 1.32 MiB)
  6.065 ms (4013 allocations: 5.49 MiB)


122896-element Vector{Int64}:
      2
      3
      5
      7
      9
     11
     13
     17
     19
     23
      ⋮
 999931
 999953
 999959
 999961
 999969
 999979
 999981
 999983
 999993

In [89]:
function isPrime7(n::Integer)
  if n==1
    return false
  end
  # get all primes up to the square root of n
  for k in getPrimes3(floor(Int,sqrt(n)))
    if n%k==0
      return false
    end
  end
  true
end

isPrime7 (generic function with 1 method)

In [90]:
@btime isPrime7(1_000_003)

  8.320 μs (107 allocations: 12.06 KiB)


true

In [91]:
n = 1_000_000_007

1000000007

In [92]:
isPrime(n)

true

In [94]:
@btime isPrime(n)
@btime isPrime2(n)
@btime isPrime3(n)
@btime isPrime4(n)
@btime isPrime5(n)
@btime isPrime6(n)
@btime isPrime7(n)

  1.836 s (5 allocations: 7.45 GiB)
  1.024 s (3 allocations: 160 bytes)
  512.446 ms (3 allocations: 160 bytes)
  32.375 μs (2 allocations: 80 bytes)
  32.375 μs (0 allocations: 0 bytes)
  16.167 μs (0 allocations: 0 bytes)
  202.167 μs (562 allocations: 213.72 KiB)


true

In [96]:
using Primes

In [98]:
@btime isprime(n)

  1.154 μs (0 allocations: 0 bytes)


true

In [49]:
n = nextprime(1_000_000_000)

1000000007

In [60]:
isPrime6(big(2)^89-1)

InterruptException: InterruptException:

In [61]:
@btime isprime(big(2)^89-1)

  9.875 μs (12 allocations: 2.66 KiB)


true